In [1]:
import gym_games
import gymnasium as gym
env = gym.make('Othello-v0' , render_mode='human')

In [17]:
obs,reward,done,_,info = env.reset()

In [31]:
env.metadata['autoplay'] = False
while True:
    print(info)
    obs,reward,done,_,info = env.step(None)


{'autoplay': True, 'turn': 2, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}
{'autoplay': False, 'turn': 1, 'action': [], 'blackSum': 26, 'whiteSum': 38}


KeyboardInterrupt: 

In [28]:
import random
env.metadata['autoplay'] = True
for i in range(10):
    action = info['action']
    if(action):
        random_choice = random.choice(info['action']) 
        obs,reward,done,_,info = env.step(random_choice)
env.render()

In [29]:
info

{'autoplay': True, 'turn': 2, 'action': [], 'blackSum': 26, 'whiteSum': 38}

In [30]:
random_choice

14